In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle


In [2]:
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.drop(["RowNumber","CustomerId","Surname"],axis=1,inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [26]:
df.isnull().sum()

CreditScore          0
Gender               0
Age                  0
Tenure               0
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
Geography_Germany    0
Geography_Spain      0
dtype: int64

In [27]:
df.duplicated().sum()

0

In [6]:
print(df["Geography"].nunique())
print(df["Geography"].value_counts())


3
Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64


In [7]:
print(df["Gender"].nunique())
print(df["Gender"].value_counts())

2
Gender
Male      5457
Female    4543
Name: count, dtype: int64


In [10]:
label_emcoder=LabelEncoder()
df["Gender"]=label_emcoder.fit_transform(df["Gender"])
df.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [15]:
onehot_encoder=OneHotEncoder(drop="first")
onehot_dataframe=onehot_encoder.fit_transform(df[["Geography"]])
onehot_dataframe

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4986 stored elements and shape (10000, 2)>

In [16]:
onehot_encoder.get_feature_names_out()

array(['Geography_Germany', 'Geography_Spain'], dtype=object)

In [20]:
data=pd.DataFrame(onehot_dataframe.toarray(),columns=onehot_encoder.get_feature_names_out())
data.head()

,Geography_Germany,Geography_Spain
0,0.0,0.0
1,0.0,1.0
2,0.0,0.0
3,0.0,0.0
4,0.0,1.0


In [23]:
df=pd.concat([df.drop("Geography",axis=1),data],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,1.0


In [24]:
with open("label_emcoder_gender.pkl","wb") as file:
    pickle.dump(label_emcoder,file)

with open("onehot_encoder_geo.pkl","wb") as file:
    pickle.dump(onehot_encoder,file)

In [25]:
x=df.drop("Exited",axis=1)
y=df["Exited"]

In [28]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [29]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [30]:
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

In [32]:
x_train.shape

(8000, 11)

In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [33]:
model=Sequential()
model.add(Dense(64,activation="relu",input_dim=11))          #first hidden layer wich include input layer
model.add(Dense(32,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                768       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2881 (11.25 KB)
Trainable params: 2881 (11.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)

In [37]:
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=["accuracy"])


In [39]:
log_dir="logs/fit"+datetime.datetime.now().strftime("%Y-%m-%d---%H-%M-%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [42]:
early_stop=EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

In [43]:
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,callbacks=[tensorflow_callback,early_stop])

Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3284 - accuracy: 0.8618 - val_loss: 0.3449 - val_accuracy: 0.8605
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3310 - accuracy: 0.8633 - val_loss: 0.3427 - val_accuracy: 0.8540
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3246 - accuracy: 0.8662 - val_loss: 0.3508 - val_accuracy: 0.8550
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3233 - accuracy: 0.8655 - val_loss: 0.3506 - val_accuracy: 0.8595
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3217 - accuracy: 0.8677 - val_loss: 0.3532 - val_accuracy: 0.8590
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3188 - accuracy: 0.8669 - val_loss: 0.3634 - val_accuracy: 0.8540
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3138 - accuracy: 0.8668 - val_loss: 0.3522 - val_accuracy: 0.8645

In [44]:
model.save("model.h5")

d:\ml\ann_churn_project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
%load_ext tensorboard

In [49]:
%tensorboard --logdir logs/fit2025-06-04---20-36-06

Reusing TensorBoard on port 6007 (pid 30360), started 0:00:16 ago. (Use '!kill 30360' to kill it.)